In [ ]:
import pandas as pd
import keywords as kwrd
import ast
from pprint import pprint
from natsort import natsorted
import json

In [ ]:
indicators_map = kwrd.indicators
indicators_map['UN_Indicators']

In [ ]:
indicators_map['UN_Indicators'] = indicators_map['UN_Indicators'].apply(lambda x: ast.literal_eval(x))

In [ ]:
test_df = pd.read_excel('output/GD61/output/6-results/mapping_.xlsx', index_col=0)[['Goal', 'Target']]
test_df.head()

In [ ]:
indicators_map = pd.merge(test_df, kwrd.indicators, how='left', on='Target').drop('Goal_y', axis=1).rename(columns={'Goal_x':'Goal'})
indicators_map.head(100)

In [ ]:
def makeindicatorsjson(resultstotal_df):
    
    indicators_dict = {"name": "sdgs", "goals": []}

    goals = natsorted(list(set(indicators_map['Goal'].values.tolist())))

    for goal in goals:
        goal_dict = {'name': goal, 'targets' : []}
        indicators_dict['goals'].append(goal_dict)
    
        for row in resultstotal_df[resultstotal_df['Goal']==goal].itertuples():
                
            target_dict={}
            target_dict['name'] = f'Target {row.Target}'
            target_dict['indicators'] = {'un':[]}

            for unind in row.UN_Indicators:
                if unind:
                    unind_dict={'name':unind}
                    target_dict['indicators']['un'].append(unind_dict)

            goal_dict['targets'].append(target_dict)      

    return indicators_dict

In [ ]:
test_indicators_dict = makeindicatorsjson(indicators_map)
print(test_indicators_dict)
with open("green_deal_indicators.json", "w") as outfile:
    json.dump(test_indicators_dict, outfile)

In [ ]:
natsorted(list(set(indicators_map['Goal'].values.tolist())))

In [ ]:
indicators_map.groupby(by='Goal')['Target'].apply([].append).reset_index()